In [14]:
dataset = "P-SET"
llm_type = "llama3.1:70b"
prompt_version=3
suffix = f"v{prompt_version}"


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np

def plot_confusion_matrices_for_column_pairs(df, gt_col,vis_columns):

    n_pairs = len(vis_columns)

    # Determine layout: square-ish grid
    n_cols = int(np.ceil(np.sqrt(n_pairs)))
    n_rows = int(np.ceil(n_pairs / n_cols))

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()
    col1 = gt_col
    for idx, col2 in enumerate(vis_columns):
        ax = axes[idx]
        
        # Drop rows with missing values in either column
        sub_df = df[[col1, col2]].dropna().copy()
        start_length = len(sub_df)
        # sub_df = sub_df[sub_df.apply(lambda x: True if ("_" not in x[col1] and '_' not in x[col2]) else False, axis=1)]
        filter_length = len(sub_df)
        # Get confusion matrix
        labels = sorted(set(sub_df[col1]) | set(sub_df[col2]))
        cm = confusion_matrix(sub_df[col1], sub_df[col2], labels=labels)

        # Plot heatmap
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, ax=ax)
        ax.set_title(f"{col1} vs \n{col2}\n strt: {start_length}\nelim.:{filter_length-start_length}", fontsize=10)
        ax.set_xlabel(col2)
        ax.set_ylabel(col1)

    # Hide any extra axes
    for j in range(len(vis_columns), len(axes)):
        fig.delaxes(axes[j])

    fig.tight_layout()
    return fig


In [16]:
from collections import Counter
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt_tab')

# Download tokenizer models if not already available
nltk.download('punkt')


def flatten_list_of_lists(nested_list):
    return [item for sublist in nested_list for item in sublist]


# Ensure required resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

def most_frequent_ngrams(sentences, N=5):
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)

    all_unigrams = []
    all_bigrams = []
    all_trigrams = []

    for sentence in sentences:
        tokens = word_tokenize(sentence.lower())
        # Remove stopwords and punctuation
        filtered_tokens = [t for t in tokens if t not in stop_words and t not in punctuation]

        all_unigrams.extend(filtered_tokens)
        all_bigrams.extend(ngrams(filtered_tokens, 2))
        all_trigrams.extend(ngrams(filtered_tokens, 3))

    # Count frequencies
    unigram_counts = Counter(all_unigrams)
    bigram_counts = Counter(all_bigrams)
    trigram_counts = Counter(all_trigrams)

    # Get top N most common
    most_common_unigram = unigram_counts.most_common(N)
    most_common_bigram = bigram_counts.most_common(N)
    most_common_trigram = trigram_counts.most_common(N)

    return {
        'most_common_unigram': most_common_unigram,
        'most_common_bigram': most_common_bigram,
        'most_common_trigram': most_common_trigram
    }


most_frequent_ngrams(["apple ate orange", "orange at babana", "apple ate", "orange at babana to"])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/asusaiyah/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/asusaiyah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/asusaiyah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/asusaiyah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'most_common_unigram': [('orange', 3),
  ('apple', 2),
  ('ate', 2),
  ('babana', 2)],
 'most_common_bigram': [(('apple', 'ate'), 2),
  (('orange', 'babana'), 2),
  (('ate', 'orange'), 1)],
 'most_common_trigram': [(('apple', 'ate', 'orange'), 1)]}

In [17]:
from glob import glob
import os
import pandas as pd
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from itertools import product
pd.set_option("display.precision", 3)


disagreement_type = "correct"

def get_time(x):
    x = np.array(x)
    # Remove NaNs
    x = x[~np.isnan(x)]
    # If less than 3 values, just return mean
    if len(x) < 3:
        return np.mean(x)
    # Remove outliers using IQR
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    mask = (x >= q1 - 1.5 * iqr) & (x <= q3 + 1.5 * iqr)
    return np.mean(x[mask])

def fixnames(ls,suffix,remove):
    LS=[]
    for s in ls:
        s = s.replace("_keyword_evidence","_Ki").replace("_example_evidence","_Ei").replace("_no_evidence","").replace("_all_evidence","KiEi").replace("Event_Name_",'').replace("_Events",'_').replace('_'+remove,"")
        s = s + "_" + suffix
        if "sentence" in s:
            s = s.replace("sentence","")
            s = s + "_Sent-SET"
        elif "document" in s:
            s = s.replace("document","")
            s = s + "_Doc-SET"
        LS.append(s)
    return LS

def get_col_suffix(keyword_input, example_input):
    col_suffix = "no"
    if keyword_input and example_input:
        col_suffix = "all"
    elif keyword_input and not example_input:
        col_suffix = "keyword"
    elif not keyword_input and example_input:
        col_suffix = "example"
    return col_suffix

LLM_dict = {}
llm_models_all = []
for keyword_input, example_input in [i for i in product([True,False],[True,False])]: 
    for input_type in ["Sent", "Doc"]:
        col_suffix = get_col_suffix(keyword_input, example_input)
        llm_models_all.append(f"LLM_Events_{col_suffix}_evidence_{input_type}")
df_both = pd.read_pickle("../exports/04b_dictionary_features.pkl")


def ispresent(a,b):
    res = []
    for i in a:
        presence = False
        for j in b:
            if (i in j) or (j in i):
                presence = True
                break
        res.append(presence)
    return res

dataset = "P-SET"
for analysis_type in ["all"]: #(M-SET, A-SET, D-SET, F-SET)
    for ET in ["Sleep"]:
        print(f"********************{ET}************************")
        llm_files = glob(f"../exports/05b_llm_{llm_type}_{dataset}_{suffix}/{ET}/*.pkl")
        print(llm_files)
        try:
            gt_file_sent = glob(f"../exports/04b_groundtruth/{dataset}/Annotated/{ET}_Sentences.pkl")[0]
            gt_file_doc = glob(f"../exports/04b_groundtruth/{dataset}/Annotated/{ET}_Documents.pkl")[0]
        except IndexError:
            print(f"No ground truth file found for {ET} in ../exports/04b_groundtruth/{dataset}/Annotated/")
            continue    
        
        llm_files = [i for i in llm_files if "True" in i]
        llm_files = [i for i in llm_files if "Sent" in i]
        for file in llm_files:
            filename = os.path.basename(file).rstrip('.pkl')
            print(filename)
            text_type = "Document" if "Document" in filename else "Sentence"
            _,attribute_requested = filename.split("_")[-2:]
            attribute_requested = eval(attribute_requested)
            suffix = "Ao" if attribute_requested else ""           
            print(suffix,attribute_requested,filename)
            
            if text_type == "Sentence":
                gt_file = gt_file_sent
                id_type = "UID"
                focus = "Sent"
                gt_column = f"Sent_gt_{ET}"
            elif text_type == "Document":
                gt_file = gt_file_doc
                id_type = "ROW_ID"
                focus = "Doc"
                gt_column = f"Doc_gt_{ET}"
            print(f"****************{text_type}**************")
            gt_df = pd.read_pickle(gt_file)
            gt_df[id_type] = gt_df[id_type].astype(str)
            df_both[id_type] = df_both[id_type].astype(str)
            gt_df = gt_df.dropna(subset=gt_column)
            if not attribute_requested:
                gt_df[gt_column] = gt_df.apply(lambda x: x[gt_column] if x['negation']==False else False,axis=1)
            
            gt_df['Lemma'] = gt_df['Lemma'].apply(lambda x: tuple(x))
            gt_df["is_keyword_present"] = gt_df["Event_Name"].apply(lambda x: 1 if ET in x else 0)
            
            id_to_gt = {row[id_type]:row[gt_column] for _,row in gt_df.iterrows()}
            id_to_negation = {row[id_type]:row['negation'] for _,row in gt_df.iterrows()}
            id_to_key_present = {row[id_type]:row[f"is_keyword_present"] for _,row in gt_df.iterrows()}
            id_to_lemma = {row[id_type]:row[f"Lemma"] for _,row in gt_df.iterrows()}
            id_to_comment = {row[id_type]:row[f"comment"] for _,row in gt_df.iterrows()}
            id_to_dict_time = {row[id_type]:row[f"Time"] for _,row in df_both.iterrows()}
            
            output_folder = f"../exports/06b_analysis/{ET}"
            os.makedirs(output_folder,exist_ok=True)
       
            
            
            df = pd.read_pickle(file)
            df["num_events_keyword"] = df["Event_Name"].apply(lambda x: len(x))
            df[f"num_{ET}_keyword"] = df["Event_Name"].apply(lambda x: len([i for i in x if i == ET]))
            df[id_type] = df[id_type].astype(str)
            llm_models = [i for i in llm_models_all if i in df]
            df[f'{ET}_time'] = df[id_type].apply(lambda x:id_to_dict_time.get(x))
            df["focus_event"] = ET
            df['negation'] = df[id_type].apply(lambda x:id_to_negation.get(x))
            df['comment'] = df[id_type].apply(lambda x:id_to_comment.get(x))
            
            
            df[gt_column] = df[id_type].apply(lambda x:id_to_gt.get(x,None))
            df[f"is_keyword_present"] = df[id_type].apply(lambda x:id_to_key_present.get(x))
            display(df.is_keyword_present.value_counts())
            print(df[[gt_column,"is_keyword_present"]].value_counts())
                        
            df[f"dict_Lemma"] = df[id_type].apply(lambda x:id_to_lemma.get(x))
            df.dropna(subset=gt_column,inplace=True)
            df[gt_column] = df[gt_column].astype(int)
            
            

            for col in llm_models:
                df[f"{col}_{ET}_time"] = df[col].apply(lambda x:x['event_detection_time'])
                
            if len(df) > 1:
                first_row = df.iloc[0]
                splittable_columns = ["Event_Name"]+[f"Event_Name_{model}" for model in llm_models]
                disagreement_dfs = []
                generated_columns = []

                for col in splittable_columns:
                    generated_column = f"{col}_{ET}"
                    df[generated_column] = df[col].apply(lambda x: 1 if ET in x else 0)
                    
                    generated_columns.append(generated_column)
                
                f1s, accs, precs, recs, psup, nsup, times, tp, tn, fp, fn = [],[],[],[],[],[],[],[],[],[],[]
                
                
                for col in generated_columns:
                    print(col)
                    df_temp = df.copy()
                    y_gt = df_temp[gt_column]
                    LLM_dict[(attribute_requested,gt_column)] = y_gt
                    preds = df_temp[col]
                    LLM_dict[(attribute_requested,col)] = preds
                    f1s.append(f1_score(y_gt, preds))
                    accs.append(accuracy_score(y_gt, preds))
                    precs.append(precision_score(y_gt, preds, zero_division=0))
                    recs.append(recall_score(y_gt, preds))   
                    psup.append(sum(y_gt))
                    nsup.append(sum(y_gt==0))
                    times.append(get_time(df_temp[f"{col.lstrip('Event_Name_')}_time"]))
                    tn_i, fp_i, fn_i, tp_i = confusion_matrix(y_gt, preds).ravel()
                    tp.append(tp_i)
                    tn.append(tn_i)
                    fp.append(fp_i)
                    fn.append(fn_i)
                    
                    
                    if "all_evidence" in col:
                        col_base = col.lstrip("Event_Name_").rstrip(f"_{ET}")
                        df[f"Text_Quote_{col_base}_{ET}"] = df.apply(lambda x:[t for t,e in zip(x[f"Text_Quote_{col_base}"],x[f"Event_Name_{col_base}"]) if e==ET],axis=1)
                        df[f"Attribute_{col_base}_{ET}"] = df.apply(lambda x:[a for a in (x[f"Attribute_{col_base}"]) if list(a.keys())[0]==ET],axis=1)
                        df[f"Event_Id_{col_base}"] = df[col_base].apply(lambda x:[ f"{eid}={en}|{1 if tq in x['text'] else 0}:{tq}" for (eid,en,tq) in zip(x['event_id'],x['event'],x['text_quotes'])])
                        df[f"Text_Quote_in_gt"] = df.apply(lambda x:ispresent(x[f"Text_Quote_{col_base}_{ET}"], x['comment'].split('|')), axis=1)

                        interesting_columns = [id_type, text_type, gt_column, 
                                               "is_keyword_present",
                                               f"Event_Name_{ET}", "comment", "negation", 
                                               col, "Text_Quote_in_gt",
                                               f"Text_Quote_{col_base}_{ET}",
                                               f"Attribute_{col_base}_{ET}"]+[i+col_base for i in ["Event_Name_","Attribute_","Text_Quote_", "Event_Id_","Order_"]] 
                        
                        
                        false_positive_samples = df[(df[col]!=df[gt_column])&(df[col]==1)]
                        false_positive_samples[interesting_columns].to_excel(f"../exports/06b_analysis/{ET}/FP_{col}.xlsx")
                        print("false_positive_keywords:",most_frequent_ngrams(flatten_list_of_lists(false_positive_samples[f"Text_Quote_{col_base}_{ET}"].to_list()),15))
                        false_negative_samples = df[(df[col]!=df[gt_column])&(df[col]==0)]
                        false_negative_samples[interesting_columns].to_excel(f"../exports/06b_analysis/{ET}/FN_{col}.xlsx")
                        # print("false_negative_keywords:",most_frequent_ngrams(flatten_list_of_lists(false_positive_samples[f"comment"].to_list()),10))
                        true_positive_samples = df[(df[col]==df[gt_column])&(df[gt_column]==1)]
                        true_positive_samples[interesting_columns].to_excel(f"../exports/06b_analysis/{ET}/TP_{col}.xlsx")
                        true_negative_samples = df[(df[col]==df[gt_column])&(df[gt_column]==0)]
                        true_negative_samples[interesting_columns].to_excel(f"../exports/06b_analysis/{ET}/TN_{col}.xlsx")
                        print("TP",len(true_positive_samples),"TN",len(true_negative_samples),"FP",len(false_positive_samples),"FN",len(false_negative_samples),"%%%%%%%%%%%%%%%%%%%%%%%")
           
                results_df = {"col_name":generated_columns, "technique":fixnames(generated_columns,suffix,ET), "pos_sup": psup, "neg_sup": nsup, "f1_score":f1s, "precision":precs, "recall":recs, "TP":tp, 'TN':tn, 'FP':fp, 'FN':fn, "time":times} 
                results = pd.DataFrame(results_df)   
                plot_confusion_matrices_for_column_pairs(df, gt_column,vis_columns=generated_columns)
                op_path = f"{output_folder}/{analysis_type}_{disagreement_type}_{filename}.xlsx"
                df.to_excel(op_path,index=False)
                print(f"file written to {op_path}")
                display(results)
                

********************Sleep************************
[]


In [5]:
false_positive_samples[text_type].tolist()

['sister remained at bedside, sleeping on cot',
 'monitor resp sts/sleep apnea',
 'wife states patient wears oral airway piece r/t sleep apnea which she will bring in during next visit (sat)',
 ',daughter in room sleeping overnt-',
 'sedation/sleep meds',
 'explain to patient about sleep apnea and how patient saturation dropped will affect the heart/body',
 'asking for sleeping aid again',
 'ccu nursing progress note 1900-0700: 3vd, preop cabgs-" am i getting a sleeping pill tonight',
 'ativan ordered for sleep per anesthesia',
 'stated that he was very tired and wanted to sleep around 9pm',
 'i stated that he would have to undergo a sleep study for an official diagnosis.patient',
 'requesting ativan for sleep',
 'social: parents and sister in most of night, parents slept in micu waiting room',
 'to facilitate sleep & help control bp',
 'patient requesting sleep aid/ "pill to relax"',
 'ccu npn 2300-0700s:"i just want to go to sleep',
 'propofol off ~0030.patient quite restless at that

In [18]:
df[f"Text_Quote_in_gt"] = df.apply(lambda x:[((i in j) or (j in i)) for (i,j) in product(x[f"Text_Quote_{col_base}_{ET}"], x['comment'])], axis=1)


In [ ]:
true_positive_samples[interesting_columns]

In [ ]:
df[(df[col]==df[gt_column])&(df[col]==1)]


In [ ]:
f"{col}_{ET}", df.columns

In [ ]:
df_temp[f"{col.lstrip('Event_Name_')}_time"]

In [ ]:
afile="../exports/06_analysis/Sleep/all_correct_Sleep_Sentences_att_True.xlsx"
amodel = "Event_Name_LLM_Events_all_evidence_Sent_Sleep"
agt = "Sent_gt_Sleep"

adf = pd.read_excel(afile)
adf[adf[amodel]!=adf[agt]][['Sentence',agt,amodel]].to_excel(f"../exports/temp_false_predictions.xlsx")


In [ ]:
id_to_gt = {row[id_type]:row[gt_column] for _,row in gt_df.iterrows()}
set(id_to_gt.values())

In [ ]:
row['Sent_ID']

In [ ]:
df.Sentence

In [ ]:
print(f"{output_folder}/{analysis_type}_{disagreement_type}_{filename}.xlsx")

In [ ]:
from pprint import pprint
pprint(df_temp[(df_temp[gt_column]==False) & (df_temp[col]==True) & (df_temp["Event_Name_Sleep"]==False)][[id_type,text_type, gt_column, col]].to_string())

In [ ]:
gt_df.columns

In [ ]:
df = pd.read_pickle(file)
llm_models = [i for i in llm_models_all if i in df]
file,llm_models, df.columns

In [ ]:
file

In [ ]:
text_type ="Document" 
file = f'../exports/05b_llm_llama3.1:70b_P-SET/Sleep/Sleep_{text_type}s_att_True.pkl'
llm_df = pd.read_pickle(file)
for llm_model in llm_models:
    llm_df['tqvalid']=llm_df.apply(lambda x:[i in x[text_type] for i in x[f"Text_Quotes_{llm_model}"]], axis=1)
    print(llm_model, llm_df['tqvalid'].explode().value_counts())
# Text_Quotes_LLM_Events_all_evidence_Sent


In [ ]:
llm_models

In [ ]:
col

In [ ]:
df.columns

In [ ]:
df[["DOCUMENT","Event_Name_LLM_Events_keyword_evidence_document_Ao","Sentence" ,"GT_Sleep"]].to_excel("../exports/temp_result.xlsx",index=False)

In [ ]:
'dictionary_Sleep_time' in df.columns, llm_models

In [ ]:
y_gt.value_counts()

In [ ]:
gt_df.Keyword.value_counts()

In [ ]:
df_dictionary = pd.read_pickle(f"../exports/03b_selected_reports_with_event_log_only_dictionary_v2/combined.pkl")

In [ ]:
df_dictionary['Events'].iloc[0]

In [ ]:
import pandas as pd
from glob import glob
ET = "Sleep"
df = pd.read_pickle(glob(f"../exports/04_groundtruth/P-SET/Generated/{ET}*.pkl")[0])
print(df.UID.nunique(), df.Sentence.nunique(), df.ROW_ID.nunique(), df.SUBJECT_ID.nunique(), df.Event_Name.value_counts())

df_exploded = df.explode(['Keyword','Lemma','Event_Name','Keyword_Position'])
df_exploded['KUID'] = df_exploded['UID'] + "_" + df_exploded['Keyword_Position'].astype(str)
print(df_exploded.Event_Name.value_counts(),df_exploded[df_exploded.Event_Name==ET]['Lemma'].value_counts())

# df[['SUBJECT_ID', 'LOS_DAYS', 'AGE']].drop_duplicates().mean()




In [ ]:
df_exploded[df_exploded.Event_Name=="Sleep"].to_excel("../exports/temp_sleep_keywords.xlsx")

In [ ]:
top10_patients = df_exploded[df_exploded.Event_Name=="Sleep"].SUBJECT_ID.unique().tolist()

In [ ]:
df[df.SUBJECT_ID.isin(top10_patients)].to_excel("../exports/temp_top10_patients_all_events.xlsx")